In [1]:
# ***Be sure to source setup.bash before launching jupyter server***
%matplotlib inline

import rospy
import rosbag
import numpy as np
import json
import os
import glob
import matplotlib.pyplot as plt
import tf_conversions
import scipy.stats as st

from matplotlib.patches import Ellipse

from custom_messages.msg import SimulationDataMsg

import sys
sys.path.append("/home/joshua/Documents/Uni/Year4/dissertation/catkin_ws/src/linefollow_gazebo/scripts/")
import CameraNetwork
import Path 
import RoadModel
import Tracer


In [2]:
def point_to_numpy(point):
    return np.array([point.x, point.y, point.z])
def vec_to_numpy(vec):
    return point_to_numpy(vec)
def quat_to_numpy(quat):
    return np.array([quat.x, quat.y, quat.z, quat.w])
def quat_to_rpy(quat):
    # note: need numpy quaternion
    return tf_conversions.transformations.euler_from_quaternion(quat)

In [3]:
experiment_defs_root = "defs"
experiment_gen_root = "gen"
specific_experiment_paths = [['calibration_circuit.json'], ['calibration_circuit_mirror.json']] # subbpath to only process 1 experiment dir or .json, not all at once
# specific_experiment_paths = [['straight_road_80m_centercam_pos3.json']]
if len(specific_experiment_paths) > 0:
    specific_experiments = [os.path.join(*p) for p in  specific_experiment_paths]


In [4]:
def get_bagfiles_for(experiment_path):
    if not experiment_path[-1].endswith('.json'):
        print("must provide .json file")
    gen_path = 'gen'
    if len(experiment_path) > 0:
        if len(experiment_path) > 1:
            gen_path = os.path.join(gen_path, os.path.join(*experiment_path[:-1]))
        exp_name = experiment_path[-1].split('.')[-2]
        gen_path = os.path.join(gen_path, exp_name)
    if os.path.isdir(gen_path):
        # collect all .bag files!
        bag_files = []
        for path, dirs, files in os.walk(gen_path):
            for f in files:
                if f.endswith('.bag'):
                    bag_file_path = os.path.join(path, f)
                    bag_files.append(bag_file_path)
        return bag_files
    else:
        print("{0} is not generated!".format(gen_path))


In [12]:
def get_summed_trace(bag, max_steps):
    total_trace = 0
    for i, msg in enumerate(bag):
        if i >= max_steps:
            break
        msg = msg.message
        ekf_cov = np.array(msg.ekf_odom.pose.covariance).reshape(6,6)[:3, :3]
        total_trace += np.sum(np.diag(ekf_cov))
    return total_trace
    
def get_final_trace(bag, max_steps):
    for i, msg in enumerate(bag):
        if i == max_steps:
            msg = msg.message
            ekf_cov = np.array(msg.ekf_odom.pose.covariance).reshape(6,6)[:3, :3]
            return np.sum(np.diag(ekf_cov))

In [13]:
def calculate_mean_summed_trace(bagfiles, max_steps):
    total, n = 0.0, 0
    for f in bagfiles:
        bag = rosbag.Bag(f)
        total += get_summed_trace(bag, max_steps)
        n += 1
    
    return total/n

def calculate_mean_final_trace(bagfiles, max_steps):
    total, n = 0.0, 0
    for f in bagfiles:
        bag = rosbag.Bag(f)
        total += get_final_trace(bag, max_steps)
        n += 1
    return total/n

In [24]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_nocam.json'])
# max_steps = get_min_number_of_steps([files])
max_steps = 120
print("Mean summed trace from covariance matrices: {0}".format(calculate_mean_summed_trace(files, max_steps)))
print("Mean final trace from EKF covariances: {0}".format(calculate_mean_final_trace(files, max_steps)))
print(max_steps)

Mean summed trace from covariance matrices: 4.08118204172
Mean final trace from EKF covariances: 0.0831503443325
120


In [21]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos1.json'])
# max_steps = get_min_number_of_steps([files])
max_steps = 120
print("Mean summed trace from covariance matrices: {0}".format(calculate_mean_summed_trace(files, max_steps)))
print("Mean final trace from EKF covariances: {0}".format(calculate_mean_final_trace(files, max_steps)))
print(max_steps)

Mean summed trace from covariance matrices: 2.30201221817
Mean final trace from EKF covariances: 0.0115630959742
120


In [22]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos2.json'])
# max_steps = get_min_number_of_steps([files])
print("Mean summed trace from covariance matrices: {0}".format(calculate_mean_summed_trace(files, max_steps)))
print("Mean final trace from EKF covariances: {0}".format(calculate_mean_final_trace(files, max_steps)))

print(max_steps)

Mean summed trace from covariance matrices: 2.06594053791
Mean final trace from EKF covariances: 0.00894009371075
120


In [23]:
# for one example for now, wrap in function later
files = get_bagfiles_for(['straight_road_80m_centercam_pos3.json'])
print("Mean summed trace from covariance matrices: {0}".format(calculate_mean_summed_trace(files, max_steps)))
print("Mean final trace from EKF covariances: {0}".format(calculate_mean_final_trace(files, max_steps)))

print(max_steps)

Mean summed trace from covariance matrices: 1.80152130858
Mean final trace from EKF covariances: 0.00819410427391
120
